In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import pandas as pd
import csv

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
#ID = "00FQb4jTyendYWaN8pK0wa"

## Uploading and displaying list of Spotify Id's for Selected Singers

In [3]:
with open('ids.csv', newline='') as f:
    reader = csv.reader(f)
    ids = list(reader)

for i in range(len(ids)):
    ids[i] = ids[i][0].replace(" ", "")
    
print(ids)

['00FQb4jTyendYWaN8pK0wa', '04gDigrS5kc9YWfZHwBETP', '06HL4z0CvFAxyc27GXpf02', '0B3N0ZINFWvizfa8bKiz4v', '0Cp8WN4V8Tu4QJQwCN5Md4', '0eDvMgVFoNV3TpwtrVCoTj', '0EFisYRi20PTADoJrifHrz', '0elWFr7TW8piilVRYJUe4P', '0GM7qgcRCORpGnfcN2tCiB', '0hCNtLu0JehylgoiP8L4Gh', '0imE3buPhAowREqCrr4CYe', '0KPX4Ucy9dk82uj4GpKesn', '0LyfQWJT6nXafLPZqxe9Of', '0NIPkIjTV8mB795yEIiPYL', '0tmwSHipWxN12fsoLcFU3B', '0u6GtibW46tFX7koQ6uNJZ', '0vR2qb8m9WHeZ5ByCbimq2', '0vRvGUQVUjytro0xpb26bs', '0xOeVMOz2fVg5BJY3N6akT', '0xRXCcSX89eobfrshSVdyu', '0XwVARXT135rw8lyw1EeWP', '0Y5tJX1MQlPlqiwlOH1tJY', '0YinUQ50QDB7ZxSCLyQ40k', '0ZED1XzwlLHW4ZaG4lOT6m', '0ZHPrnImGh4re3TbSNkoZl', '15UsOTVnJzReFVN1VCnxy4', '19ra5tSw0tWufvUp8GotLo', '1Bl6wpkWCQ4KVgnASpvzzA', '1cNDP5yjU5vjeR8qMf4grg', '1iNqsUDUraNWrj00bqssQG', '1lmU3giNF3CSbkVSQmLpHQ', '1mcTU81TzQhprhouKaTkpq', '1MIVXf74SZHmTIp4V4paH4', '1o2NpYGqHiCq7FoiYdyd1x', '1r4hJ1h58CWwUQe3MxPuau', '1SupJlEpv7RS2tPNRaHViT', '1uNFoZAHBGtllmzznpCI3s', '1vCWHaC5f2uS3yhpwWbIA6', '1vyhD5VmyZ

## Python Script for Querying Artist Voice Type from WikiData


In [4]:
def voice_query_sparql(ID):
    sparql.setQuery("""
    SELECT
      ?person ?personLabel
      ?spotifyId ?spotifyIdLabel
      ?voiceType ?voiceTypeLabel
    WHERE 
    {
      ?person wdt:P1902 ?spotifyId.
      ?person wdt:P412 ?voiceType.
      ?person wdt:P1902 '""" + ID + """'.      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'voiceType.value' in results_df.columns):
        return results_df[['spotifyIdLabel.value','voiceTypeLabel.value']]
    return pd.DataFrame()

In [5]:
artist_voice = pd.DataFrame(columns = ['Artist_Id', 'Voice_Type'])

for i in range(len(ids)):
    df = voice_query_sparql(ids[i])
    df = df.rename(columns={"spotifyIdLabel.value" : "Artist_Id", "voiceTypeLabel.value": "Voice_Type"})
    artist_voice = artist_voice.append(df)

artist_voice = artist_voice.reset_index(drop=True)
artist_voice = artist_voice.drop_duplicates()

voice = artist_voice['Voice_Type'].drop_duplicates().reset_index(drop=True)

print("ARTIST VOICE TABLE:")
display(artist_voice)

print("VOICE TABLE:")
display(voice)

ARTIST VOICE TABLE:


,Artist_Id,Voice_Type
0,00FQb4jTyendYWaN8pK0wa,contralto
1,06HL4z0CvFAxyc27GXpf02,mezzo-soprano
2,0Cp8WN4V8Tu4QJQwCN5Md4,soprano
3,0ZHPrnImGh4re3TbSNkoZl,tenor
4,19ra5tSw0tWufvUp8GotLo,tenor
5,19ra5tSw0tWufvUp8GotLo,tenore di grazia
6,1uNFoZAHBGtllmzznpCI3s,tenor
7,1vyhD5VmyZ7KMfW5gqLgo5,baritone
8,2FXC3k01G6Gw61bmprjgqS,tenor
9,2wY79sveU1sp5g7SokKOiI,countertenor


VOICE TABLE:


0           contralto
1       mezzo-soprano
2             soprano
3               tenor
4    tenore di grazia
5            baritone
6        countertenor
7      spinto soprano
Name: Voice_Type, dtype: object

## Python Script for Querying Artist Age from WikiData

In [6]:
def age_query_sparql(ID):
    sparql.setQuery("""
    SELECT
      ?person ?personLabel
      ?spotifyId ?spotifyIdLabel
      ?age ?ageLabel
    WHERE 
    {
      ?person wdt:P1902 ?spotifyId.
      ?person wdt:P569 ?birth_date.  
      BIND(year(now()) - year(?birth_date) AS ?age)
      ?person wdt:P1902 '"""+ID+"""'.      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'age.value' in results_df.columns):
        return results_df[['spotifyIdLabel.value','age.value']]
    return pd.DataFrame()

In [7]:
artist_age = pd.DataFrame(columns = ['Artist_Id', 'Age'])

for i in range(len(ids)):
    df = age_query_sparql(ids[i])
    df = df.rename(columns={"spotifyIdLabel.value" : "Artist_Id", "age.value": "Age"})
    artist_age = artist_age.append(df)

artist_age = artist_age.reset_index(drop=True)
artist_age = artist_age.drop_duplicates()

print("ARTIST AGES TABLE:")
display(artist_age)

ARTIST AGES TABLE:


,Artist_Id,Age
0,00FQb4jTyendYWaN8pK0wa,35
1,06HL4z0CvFAxyc27GXpf02,31
2,0B3N0ZINFWvizfa8bKiz4v,23
3,0Cp8WN4V8Tu4QJQwCN5Md4,20
4,0eDvMgVFoNV3TpwtrVCoTj,21
...,...,...
99,790FomKkXshlbRYZFtlgla,29
100,7jVv8c5Fj3E9VhNjxT4snq,21
101,7n2wHs1TKAczGzO7Dd2rGr,22
102,7pbDxGE6nQSZVfiFdq9lOL,22


## Python Script for Querying Instruments from WikiData

In [8]:
def instrument_query(ID):
    url = 'https://query.wikidata.org/sparql'
    query = """
    SELECT 
      ?person ?personLabel ?Artist_Id ?instrumentLabel
    WHERE {
      ?person wdt:P1902 '""" + ID +"""'
      OPTIONAL{?person wdt:P1902 ?Artist_Id .}
      OPTIONAL{?person wdt:P1303 ?instrument .}
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    r = requests.get(url, params = {'format': 'json', 'query': query})
    print(ID, ": ", r.status_code)
    if(r.status_code == 200):
        data = r.json()
        results_df = pd.json_normalize(data['results']['bindings'])
        if(not results_df.empty and 'instrumentLabel.value' in results_df.columns):
            return results_df[['Artist_Id.value','instrumentLabel.value']]
        return pd.DataFrame()
    
def instrument_query_sparql(ID):
    sparql.setQuery("""
    SELECT 
        ?person ?personLabel ?Artist_Id ?instrumentLabel
    WHERE {
        ?person wdt:P1902 '""" + ID + """'
    OPTIONAL{?person wdt:P1902 ?Artist_Id .}
    OPTIONAL{?person wdt:P1303 ?instrument .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'instrumentLabel.value' in results_df.columns):
        return results_df[['Artist_Id.value','instrumentLabel.value']]
    return pd.DataFrame()

In [9]:
artist_instrument = pd.DataFrame(columns = ['Artist_Id', 'Instrument'])

for i in range(len(ids)):
    df = instrument_query_sparql(ids[i])
    df = df.rename(columns={"Artist_Id.value" : "Artist_Id", "instrumentLabel.value": "Instrument"})
    artist_instrument = artist_instrument.append(df)

artist_instrument = artist_instrument.reset_index(drop=True)
artist_instrument = artist_instrument.drop_duplicates()

instrument = artist_instrument['Instrument'].drop_duplicates().reset_index(drop=True)

print("ARTIST INSTRUMENTS TABLE:")
display(artist_instrument)

print("INSTRUMENTS TABLE:")
display(instrument)

ARTIST INSTRUMENTS TABLE:


,Artist_Id,Instrument
0,00FQb4jTyendYWaN8pK0wa,guitar
1,00FQb4jTyendYWaN8pK0wa,singing
2,00FQb4jTyendYWaN8pK0wa,voice
3,06HL4z0CvFAxyc27GXpf02,piano
4,06HL4z0CvFAxyc27GXpf02,guitar
...,...,...
140,7n2wHs1TKAczGzO7Dd2rGr,piano
141,7n2wHs1TKAczGzO7Dd2rGr,guitar
142,7n2wHs1TKAczGzO7Dd2rGr,keyboard instrument
143,7n2wHs1TKAczGzO7Dd2rGr,voice


INSTRUMENTS TABLE:


0                        guitar
1                       singing
2                         voice
3                         piano
4                          drum
5               acoustic guitar
6                       ukulele
7               electric guitar
8                         banjo
9                 upright piano
10          keyboard instrument
11                  synthesizer
12                    FL Studio
13                    Logic Pro
14    digital audio workstation
15     acoustic-electric guitar
16                       Vocals
17                     drum kit
18                      sampler
19                 drum machine
20                MIDI keyboard
21                        flute
22                  bass guitar
23        percussion instrument
24                   Bajo sexto
25                  Bajo quinto
Name: Instrument, dtype: object

## Python Script for Querying Awards from WikiData

In [10]:
def award_query_sparql(ID):
    sparql.setQuery("""
    SELECT 
        ?person ?personLabel ?Artist_Id ?Award ?AwardLabel
    WHERE {
        ?person wdt:P1902 '""" + ID + """'
    OPTIONAL{?person wdt:P1902 ?Artist_Id .}
    OPTIONAL{?person wdt:P166 ?Award .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'AwardLabel.value' in results_df.columns):
        return results_df[['Artist_Id.value','AwardLabel.value']]
    return pd.DataFrame()

In [11]:
artist_award = pd.DataFrame(columns=['Artist_Id','Award'])

for i in range(len(ids)):
    df = award_query_sparql(ids[i])
    df = df.rename(columns={"Artist_Id.value" : "Artist_Id", "AwardLabel.value": "Award"})
    artist_award = artist_award.append(df)

artist_award = artist_award.reset_index(drop=True)
artist_award = artist_award.drop_duplicates()

awards = artist_award['Award'].drop_duplicates().reset_index(drop=True)

print("ARTIST AWARDS TABLE:")
display(artist_award)

print("AWARDS TABLE:")
display(awards)

ARTIST AWARDS TABLE:


,Artist_Id,Award
0,04gDigrS5kc9YWfZHwBETP,MTV Europe Music Award for Best New Act
1,04gDigrS5kc9YWfZHwBETP,Grammy Award for Best New Artist
2,06HL4z0CvFAxyc27GXpf02,MTV Video Music Award for Best Visual Effects
3,06HL4z0CvFAxyc27GXpf02,Grammy Award for Best Song Written for Visual ...
4,06HL4z0CvFAxyc27GXpf02,MTV Video Music Award for Best Pop Video
...,...,...
190,7jVv8c5Fj3E9VhNjxT4snq,Grammy Award for Best Music Video
191,7jVv8c5Fj3E9VhNjxT4snq,American Music Award for Favorite Rap/Hip-Hop ...
192,7jVv8c5Fj3E9VhNjxT4snq,Teen Choice Award for Choice Music – R&B/Hip-H...
193,7n2wHs1TKAczGzO7Dd2rGr,Canada's Walk of Fame


AWARDS TABLE:


0                MTV Europe Music Award for Best New Act
1                       Grammy Award for Best New Artist
2          MTV Video Music Award for Best Visual Effects
3      Grammy Award for Best Song Written for Visual ...
4               MTV Video Music Award for Best Pop Video
                             ...                        
139                                 Primetime Emmy Award
140               Latin Grammy Award for Best New Artist
141      Grammy Award for Best Pop Duo/Group Performance
142    Teen Choice Award for Choice Music – R&B/Hip-H...
143                                          Sound of...
Name: Award, Length: 144, dtype: object

## Python Script for Querying Influenced By from WikiData

In [12]:
def influenced_query_sparql(ID):
    sparql.setQuery("""
    SELECT
      ?person ?personLabel
      ?spotifyId ?spotifyIdLabel
      ?influencedBy ?influencedByLabel
      ?influencedId ?influencedIdLabel
    WHERE 
    {
      ?person wdt:P1902 ?spotifyId.
      ?person wdt:P737 ?influencedBy.
      ?person wdt:P1902 '""" + ID + """'.
      ?influencedBy wdt:P1902 ?influencedId.      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'influencedBy.value' in results_df.columns):
        return results_df[['spotifyIdLabel.value','influencedIdLabel.value']]
    return pd.DataFrame()

In [13]:
artist_influenced = pd.DataFrame(columns=['Artist_Id','Influenced_By'])

for i in range(len(ids)):
    df = influenced_query_sparql(ids[i])
    df = df.rename(columns={"spotifyIdLabel.value" : "Artist_Id", "influencedIdLabel.value": "Influenced_By"})
    artist_influenced = artist_influenced.append(df)

artist_influenced = artist_influenced.reset_index(drop=True)
artist_influenced = artist_influenced.drop_duplicates()

artist_influenced = artist_influenced.loc[artist_influenced['Influenced_By'].isin(ids)]

print("ARTIST INFLUENCED TABLE:")
display(artist_influenced)

ARTIST INFLUENCED TABLE:


,Artist_Id,Influenced_By
9,2wY79sveU1sp5g7SokKOiI,6XpaIBNiVzIetEPCWDvAFP
12,2wY79sveU1sp5g7SokKOiI,5WUlDfRSoLAfcVSX1WnrxN


## Python Script for Querying Member Of from WikiData

In [14]:
def member_query_sparql(ID):
    sparql.setQuery("""
    SELECT
      ?person ?personLabel
      ?spotifyId ?spotifyIdLabel
      ?memberOf ?memberOfLabel
      ?memberOfId ?memberOfIdLabel
    WHERE 
    {
      ?person wdt:P1902 ?spotifyId.
      ?person wdt:P463 ?memberOf.
      ?person wdt:P1902 '""" + ID + """'.
      ?memberOf wdt:P1902 ?memberOfId.      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'memberOf.value' in results_df.columns):
        return results_df[['spotifyIdLabel.value','memberOfIdLabel.value']]
    return pd.DataFrame()

In [15]:
artist_member = pd.DataFrame(columns=['Artist_Id','Member_Of'])

for i in range(len(ids)):
    df = member_query_sparql(ids[i])
    df = df.rename(columns={"spotifyIdLabel.value" : "Artist_Id", "memberOfIdLabel.value": "Member_Of"})
    artist_member = artist_member.append(df)

artist_member = artist_member.reset_index(drop=True)
artist_member = artist_member.drop_duplicates()

artist_member = artist_member.loc[artist_member['Member_Of'].isin(ids)]

print("ARTIST MEMBER TABLE:")
display(artist_member)

ARTIST MEMBER TABLE:


,Artist_Id,Member_Of


## Python Script for Querying Student Of from WikiData

In [16]:
def student_query_sparql(ID):
    sparql.setQuery("""
    SELECT
      ?person ?personLabel
      ?spotifyId ?spotifyIdLabel
      ?studentOf ?studentOfLabel
      ?studentOfId ?studentOfIdLabel
    WHERE 
    {
      ?person wdt:P1902 ?spotifyId.
      ?person wdt:P1066 ?studentOf.
      ?person wdt:P1902 '""" + ID + """'.
      ?studentOf wdt:P1902 ?studentOfId.      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    if(not results_df.empty and 'studentOf.value' in results_df.columns):
        return results_df[['spotifyIdLabel.value','studentOfIdLabel.value']]
    return pd.DataFrame()

In [17]:
artist_student = pd.DataFrame(columns=['Artist_Id','Student_Of'])

for i in range(len(ids)):
    df = student_query_sparql(ids[i])
    df = df.rename(columns={"spotifyIdLabel.value" : "Artist_Id", "memberOfIdLabel.value": "Member_Of"})
    artist_student = artist_student.append(df)

artist_student = artist_student.reset_index(drop=True)
artist_student = artist_student.drop_duplicates()

artist_student = artist_student.loc[artist_student['Student_Of'].isin(ids)]

print("ARTIST STUDENT TABLE:")
display(artist_student)

ARTIST STUDENT TABLE:


,Artist_Id,Student_Of


## Creating All Necessary CSV's

In [21]:
voice.to_csv('voice.csv', index=False, header=True)
artist_voice.to_csv('artist_voice.csv', index=False, header=True)

artist_age.to_csv('artist_age.csv', index=False,header=True)

instrument.to_csv('instrument.csv', index=False,header=True)
artist_instrument.to_csv('artist_inst.csv', index=False,header=True)

artist_award.to_csv('artist_award.csv', index=False,header=True)
awards.to_csv('award.csv', index=False, header=True)

artist_influenced.to_csv('artist_inf.csv', index=False,header=True)

artist_member.to_csv('artist_mem.csv', index=False,header=True)

artist_student.to_csv('artist_stud.csv', index=False,header=True)